In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import time



In [9]:
url='https://www.udemy.com/courses/search/?src=ukw&q=courses'
page=requests.get(url)
soup=BeautifulSoup(page.text,"html")
print(soup)


<!DOCTYPE html>

<html lang="en-us">
<head>
<title>Search results | Udemy</title>
<!-- OneTrust Cookies Consent Notice start for *.udemy.com display_type is web-->
<script nonce="2A0hyGZgbDrY+6Rz6DbJDNEUxrWe+/BnpIyYREJEGHY=" type="text/javascript">
        // Note: must be called before google tag manager is initialized in OptanonWrapper()
        if (typeof window.dataLayer == 'undefined') {
            window.dataLayer = window.dataLayer || [];
        }

        function callGtag() {
            window.dataLayer.push(arguments);
        }

        var GRANTED = 'granted';
        var DENIED = 'denied';

        window.optOutConsent = {
            analytics_storage: GRANTED,
            functionality_storage: GRANTED,
            personalization_storage: GRANTED,
            security_storage: GRANTED,
            ad_storage: GRANTED,
            ad_user_data: GRANTED,
            ad_personalization: GRANTED,
        }

        window.optInConsent = {
            analytics_storage: 

In [22]:
soup.find_all("div", class_="course-card--main-content")
print(soup)
courses=soup.find_all("div", class_ = "course-product-card--quick-view-box-container--HzFAc")
for course in courses:
    title=course.find("div", class_="course-card--course-title--2f7tE").text
    print(title)    


<!DOCTYPE html>

<html lang="en-us">
<head>
<title>Search results | Udemy</title>
<!-- OneTrust Cookies Consent Notice start for *.udemy.com display_type is web-->
<script nonce="2A0hyGZgbDrY+6Rz6DbJDNEUxrWe+/BnpIyYREJEGHY=" type="text/javascript">
        // Note: must be called before google tag manager is initialized in OptanonWrapper()
        if (typeof window.dataLayer == 'undefined') {
            window.dataLayer = window.dataLayer || [];
        }

        function callGtag() {
            window.dataLayer.push(arguments);
        }

        var GRANTED = 'granted';
        var DENIED = 'denied';

        window.optOutConsent = {
            analytics_storage: GRANTED,
            functionality_storage: GRANTED,
            personalization_storage: GRANTED,
            security_storage: GRANTED,
            ad_storage: GRANTED,
            ad_user_data: GRANTED,
            ad_personalization: GRANTED,
        }

        window.optInConsent = {
            analytics_storage: 

In [23]:
import requests
from bs4 import BeautifulSoup
import csv
import time

HEADERS = {"User-Agent": "Mozilla/5.0"}


def scrape_course_details(url):
    """Fetch duration and difficulty from a course detail page."""
    try:
        resp = requests.get(url, headers=HEADERS)
        if resp.status_code != 200:
            return None, None
        soup = BeautifulSoup(resp.content, "html.parser")

        # Duration (e.g., "40.5 total hours")
        duration_el = soup.find(
            "span", string=lambda t: t and "total hours" in t.lower()
        )
        duration = duration_el.text.strip() if duration_el else None

        # Difficulty (e.g., "Beginner", "Intermediate", "All Levels")
        diff_el = soup.find(
            "span",
            string=lambda t: t
            and t.lower() in ["beginner", "intermediate", "advanced", "all levels"],
        )
        difficulty = diff_el.text.strip() if diff_el else None

        return duration, difficulty
    except Exception:
        return None, None


def scrape_udemy(search_url, pages=1, output_file="udemy_courses.csv"):
    results = []
    for page in range(1, pages + 1):
        url = f"{search_url}&p={page}"
        resp = requests.get(url, headers=HEADERS)
        if resp.status_code != 200:
            print(f"Failed to fetch page {page}")
            continue

        soup = BeautifulSoup(resp.content, "html.parser")
        courses = soup.find_all("div", class_="course-card--main-content")

        for c in courses:
            title_el = c.find("h3")
            title = title_el.text.strip() if title_el else None

            link_el = c.find("a", href=True)
            course_url = "https://www.udemy.com" + link_el["href"] if link_el else None

            instructor_el = c.find("div", class_="course-card--instructor-list")
            instructor = instructor_el.text.strip() if instructor_el else None

            rating_el = c.find("span", class_="star-rating--rating-number")
            rating = rating_el.text.strip() if rating_el else None

            reviews_el = c.find("span", class_="course-card--reviews-text")
            reviews = reviews_el.text.strip() if reviews_el else None

            price_el = c.find("div", class_="price-text--price-part")
            price = price_el.text.strip() if price_el else None

            # Fetch detail page for duration & difficulty
            duration, difficulty = (None, None)
            if course_url:
                duration, difficulty = scrape_course_details(course_url)
                time.sleep(1)  # polite delay

            results.append(
                {
                    "course_title": title,
                    "platform": "Udemy",
                    "category": "courses",
                    "price": price,
                    "duration": duration,
                    "difficulty": difficulty,
                    "instructor": instructor,
                    "rating": rating,
                    "num_reviews": reviews,
                    "url": course_url,
                }
            )

        time.sleep(1)

    # Save to CSV
    fieldnames = [
        "course_title",
        "platform",
        "category",
        "price",
        "duration",
        "difficulty",
        "instructor",
        "rating",
        "num_reviews",
        "url",
    ]
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)

    print(f"Scraped {len(results)} courses into {output_file}")


# Example usage
if __name__ == "__main__":
    scrape_udemy(
        "https://www.udemy.com/courses/search/?src=ukw&q=courses&sort=highest-rated",
        pages=1,
    )

Failed to fetch page 1
Scraped 0 courses into udemy_courses.csv


In [10]:
def scrape_udemy(search_query, pages=1, output_file="udemy_courses.csv"):
    base_url = "https://www.udemy.com/courses/search/"
    results = []

    for page in range(1, pages + 1):
        params = {"q": search_query, "p": page}
        response = requests.get(
            base_url, params=params, headers={"User-Agent": "Mozilla/5.0"}
        )
        if response.status_code != 200:
            print(f"Failed to fetch page {page}")
            continue

        soup = BeautifulSoup(response.text, "html.parser")

        # Each course card
        courses = soup.select("div.course-card--main-content")
        for course in courses:
            title_el = course.select_one("h3")
            title = title_el.text.strip() if title_el else None

            url_el = course.select_one("a")
            url = "https://www.udemy.com" + url_el["href"] if url_el else None

            instructor_el = course.select_one(".course-card--instructor-list")
            instructor = instructor_el.text.strip() if instructor_el else None

            rating_el = course.select_one(".star-rating--rating-number")
            rating = rating_el.text.strip() if rating_el else None

            reviews_el = course.select_one(".course-card--reviews-text")
            reviews = reviews_el.text.strip() if reviews_el else None

            price_el = course.select_one(".price-text--price-part")
            price = price_el.text.strip() if price_el else None

            results.append(
                {
                    "course_title": title,
                    "platform": "Udemy",
                    "category": search_query,
                    "price": price,
                    "duration": None,  # not available on listing
                    "difficulty": None,  # not available on listing
                    "instructor": instructor,
                    "rating": rating,
                    "num_reviews": reviews,
                    "url": url,
                }
            )

        time.sleep(1)  # polite delay

    # Save to CSV
    fieldnames = [
        "course_title",
        "platform",
        "category",
        "price",
        "duration",
        "difficulty",
        "instructor",
        "rating",
        "num_reviews",
        "url",
    ]
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)

    print(f"Scraped {len(results)} courses into {output_file}")


# Example usage
if __name__ == "__main__":
    scrape_udemy("courses", pages=2)

Failed to fetch page 1
Failed to fetch page 2
Scraped 0 courses into udemy_courses.csv
